<a href="https://colab.research.google.com/github/borgesf/Formula_Magica_Greenblatt/blob/main/Magic_Formula_Greenblatt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fórmula Mágica - Joel Greenblat

O objetivo desse notebook é gerar um *ranking* de ações da Bovespa, seguindo critérios de liquidez diária, ROIC e P/L. A fonte dos dados é o site [Fundamentus](https://www.fundamentus.com.br).


### Autor: Filipe Borges (filipe.borges@ntnu.no)
### [Fonte do material: Tutorial em Vídeo - Canal Código Quant](https://www.youtube.com/watch?v=e_ZRDG4F4ZA&list=PLCAhGm8nJ9CBn51o0x3j1p1LuMRqpeqCy&index=8)
### Data da Versão: 14/06/2021

## 1 - Importando Bibliotecas e Carregando Dados

In [2]:
import pandas as pd
import requests

url = 'http://www.fundamentus.com.br/resultado.php'
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

# Pre-processamento dos dados: troca ',' por '.' em alguns decimais, e converte strings para float

for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype(float) / 100

## Filtros de liquidez e "qualidade" - P/L e ROE

In [3]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - P/L (>0) e ROE
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['P/L'] = df[df['P/L'] > 0].sort_values(by=['P/L'])['Papel'][:150].values
ranking['ROE'] = df.sort_values(by=['ROE'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='P/L', values='position')
aux_b = ranking.pivot_table(columns='ROE', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:15]

MRFG3      7
TASA3     13
TASA4     13
CSNA3     32
TAEE3     40
TAEE4     40
TAEE11    40
ETER3     48
CYRE3     50
LUPA3     53
RAPT4     53
PLPL3     56
BEEF3     56
TRPL4     57
JBSS3     58
dtype: int64

## Filtros de liquidez e "qualidade" alternativo - EV/EBIT e ROIC (NÃO VALE PARA FINANCEIRAS E BANCOS!!!)

In [4]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - P/L (>0) e ROE
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='EV/EBIT', values='position')
aux_b = ranking.pivot_table(columns='ROIC', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:15]

PSSA3      6
WIZS3     11
CMIN3     20
MRFG3     21
PLPL3     22
VALE3     24
AURA33    24
TASA3     27
TASA4     28
UNIP6     48
BRKM5     51
CSNA3     51
BRKM3     52
ALUP11    52
BEEF3     52
dtype: int64